In [ ]:
import os
import psycopg2
from datetime import datetime
from dotenv import load_dotenv


Charger automatiquement le fichier .env depuis le dossier courant

In [ ]:
load_dotenv()


Récupérer les variables d'environnement pour les chemins

In [ ]:
BASE_DIR = os.getenv("BASE_DIR")
LOG_DIR = os.getenv("LOG_DIR")

def ensure_log_dir():
    if not os.path.exists(LOG_DIR):
        os.makedirs(LOG_DIR)

def get_log_file():
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    return os.path.join(LOG_DIR, f"create_tables_{now}.log")

class Logger:
    def __init__(self, filepath):
        self.filepath = filepath
        self.log_lines = []

    def print(self, msg):
        print(msg)
        self.log_lines.append(msg)

    def save(self):
        with open(self.filepath, "w", encoding="utf-8") as f:
            f.write("\n".join(self.log_lines))

def connect_db():
    return psycopg2.connect(
        dbname=os.getenv("POSTGRES_DB"),
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        host=os.getenv("POSTGRES_HOST"),
        port=os.getenv("POSTGRES_PORT")
    )

def create_tables(cur, log):
    log.print("🔄 Suppression des tables existantes si elles existent...")

    cur.execute("DROP TABLE IF EXISTS avis_trustpilot;")
    cur.execute("DROP TABLE IF EXISTS societe;")
    log.print("✅ Tables supprimées (si elles existaient).")

    log.print("🔄 Création de la table societe...")